In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
def read_csv_file(filename):
    df = pd.read_csv(filename)
    print(df.info())
    return  df

In [ ]:
df = read_csv_file('train.csv')
df.head()

In [ ]:
fil_cols = [ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st', 'personal_status', 'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type', 'telephone' , 'foreigner' , 'serial number' , 'duration_month',  'poi' , 'resident_since' , 'age', 'credits_no', 'liables', 'credit_amount']
len(fil_cols)
new_fil_cols =[ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st',  'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type' , 'foreigner' , 'serial number' , 'duration_month',  'poi' , 'resident_since' , 'age', 'credits_no', 'liables', 'credit_amount']
new_val_cols =[ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st',  'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type' , 'foreigner' , 'serial number' , 'duration_month',  'poi' , 'resident_since' , 'age', 'credits_no', 'liables']

val_cols = [ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st', 'personal_status', 'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type', 'telephone' , 'foreigner' , 'serial number' , 'duration_month',  'poi' , 'resident_since' , 'age', 'credits_no', 'liables']
obj_cols = [ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st', 'personal_status', 'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type', 'telephone' , 'foreigner' ]

new_obj_cols = [ 'account_info' , 'credit_history' , 'purpose' , 'savings_account', 'employment_st',  'gurantors', 'property_type' ,'installment_type' ,'housing_type' ,'job_type' , 'foreigner' ]

In [ ]:
def reorder_columns(fil_cols, df):
    df =df[fil_cols]
    return df

In [ ]:
df =reorder_columns(fil_cols, df)

In [ ]:
df.head()

In [ ]:
df['credit_amount'].nunique()

In [ ]:
def drop_columns(df , columns):
    new_df = df.drop(columns =columns, axis = 1)
    return new_df
columns_to_be_dropped = ['serial number','telephone','personal_status']
new_df = drop_columns(df, columns_to_be_dropped)


In [ ]:
new_df = pd.get_dummies(new_df, columns = new_obj_cols)

In [ ]:
new_df.head()

In [ ]:
new_df = new_df.sample(frac=1)

In [ ]:
def getFeatures(new_df):
    labels = new_df['credit_amount']
    X = drop_columns(new_df,'credit_amount')
    return np.array(X), np.array(labels)


In [ ]:
X , y = getFeatures(new_df)

In [ ]:
X.shape, y.shape

In [ ]:
X[0] ,y[0]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid , y_train, y_valid = train_test_split(X, y, test_size =0.1, shuffle=True)

In [ ]:
X_train.shape , y_train.shape , X_valid.shape , y_valid.shape

In [ ]:
# from sklearn.decomposition import PCA
# pca= PCA()
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_valid = pca.transform(X_valid)

# X_train.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
scale.fit(X_train)
X_train = scale.transform(X_train)
X_valid = scale.transform(X_valid)
X_train.shape, X_valid.shape
# y_valid_scaled = scale_y.transform(y_valid_new)
# #y_valid = minmax(y_valid)


In [ ]:
# y_train_new = y_train.reshape(-1,1)
# y_valid_new = y_valid.reshape(-1,1)
# y_train_new.shape,y_valid_new.shape
# from sklearn.preprocessing import MinMaxScaler
# scale_y = MinMaxScaler()
# scale_y.fit(y_train_new)
# y_train_scaled = scale_y.transform(y_train_new)
# y_valid_scaled = scale_y.transform(y_valid_new)
# #y_valid = minmax(y_valid)


In [ ]:
# y_train_scaled[3] , y_valid_scaled[3]

In [ ]:
import keras
from keras.layers import BatchNormalization
from keras.layers.core import Flatten,Dense,Dropout, Activation, Lambda
from keras.models import Model, Sequential
from keras.optimizers import Adam,SGD
from keras.callbacks import TensorBoard

In [ ]:
from keras import backend as K

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return (K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)))

In [ ]:
model = Sequential()
#model.add(Lambda(lambda x: x , input_shape = (24)))
#model.add(Lambda(lambda x: x+0.1 , input_shape = (64,64,3)))
#model.add(Flatten())
model.add(BatchNormalization(input_shape=(54,)))
model.add(Dense(100))
model.add(Activation('relu'))


model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(optimizer = 'Adam' , loss = root_mean_squared_error)

In [ ]:
model.fit(X_train, y_train, epochs=500, verbose =1, validation_data=(X_valid, y_valid), batch_size=32)

In [ ]:
model.save('./reg_model_2.h5')

In [ ]:
test_df = pd.read_csv('./test.csv')

In [ ]:
test_df = reorder_columns(val_cols,test_df)

In [ ]:
new_test_df = pd.get_dummies(test_df, columns = obj_cols)

In [ ]:
new_test_df = drop_columns(new_test_df, columns_to_be_dropped)


In [ ]:
X_test = np.array(new_test_df)
X_test.shape

In [ ]:
X_test_scaled = pca.transform(X_test)
X_test_scaled.shape

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#y_pred = scale_y.inverse_transform(y_pred)

In [ ]:
def write_csv(pred_test, filename):
    df = pd.read_csv('sample_reg.csv')
    df['credit_amount'] = pred_test
    df.to_csv(filename,index=False)

In [ ]:
write_csv(y_pred, 'regression_v3.csv')

In [ ]:
def get_classification_labels(row):
#for row in new_df.itertuples():
    #print(row.credit_amount)
    if(row < 1500):
        return 3
    elif(row >= 1500 and row < 4000):
        return 2
    elif (row >= 4000 and row < 20000):
        return 1

In [ ]:
def write_csv_new( filename):
    df = pd.read_csv('regression_v3.csv')
    df['cluster_number'] = df['credit_amount'].apply(get_classification_labels)
    df = df.drop(columns=['credit_amount'])
    df.to_csv(filename,index=False)

In [ ]:
write_csv_new('classification_v2.csv')